In [2]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as py
import seaborn as sns
import plotly.express as px

In [3]:
df = pd.read_csv("hospital_supply_chain.csv")

In [4]:
df.head()

,Item Name,Supplier,Quantity,Cost Per Unit,Total Cost,Expiry Date,Storage Location,Stock Status,Order Date,Delivery Date
0,Alone Sanitizers,MedSupplies,144,487.83,70247.52,2026-12-03,Main Hospital,Low Stock,2022-12-05,2024-12-29
1,Foot Diagnostic Tools,CareWell,152,334.54,50850.08,2027-05-01,Clinic Branch,Available,2023-03-16,2023-04-13
2,Air Diagnostic Tools,CareWell,258,481.12,124128.96,2026-12-29,Emergency Storage,Out of Stock,2024-08-13,NaN
3,Child Diagnostic Tools,MediCorp,234,405.56,94901.04,2025-10-11,Warehouse B,Low Stock,2022-09-14,2025-02-25
4,Exactly Medicine,MediCorp,57,284.75,16230.75,2027-04-17,Main Hospital,Available,2022-10-14,2024-01-06


In [5]:
df.info

<bound method DataFrame.info of                                      Item Name     Supplier  Quantity  \
0                             Alone Sanitizers  MedSupplies       144   
1                        Foot Diagnostic Tools     CareWell       152   
2                         Air Diagnostic Tools     CareWell       258   
3                       Child Diagnostic Tools     MediCorp       234   
4                             Exactly Medicine     MediCorp        57   
...                                        ...          ...       ...   
7995                           Hour Sanitizers  HealthFirst        98   
7996  Experience Personal Protective Equipment     MediCorp       243   
7997                   Play Surgical Equipment  HealthFirst       124   
7998     Purpose Personal Protective Equipment   PharmaPlus       278   
7999                     Term Diagnostic Tools     MediCorp        59   

      Cost Per Unit  Total Cost Expiry Date   Storage Location  Stock Status  \
0          

In [6]:
df.describe

<bound method NDFrame.describe of                                      Item Name     Supplier  Quantity  \
0                             Alone Sanitizers  MedSupplies       144   
1                        Foot Diagnostic Tools     CareWell       152   
2                         Air Diagnostic Tools     CareWell       258   
3                       Child Diagnostic Tools     MediCorp       234   
4                             Exactly Medicine     MediCorp        57   
...                                        ...          ...       ...   
7995                           Hour Sanitizers  HealthFirst        98   
7996  Experience Personal Protective Equipment     MediCorp       243   
7997                   Play Surgical Equipment  HealthFirst       124   
7998     Purpose Personal Protective Equipment   PharmaPlus       278   
7999                     Term Diagnostic Tools     MediCorp        59   

      Cost Per Unit  Total Cost Expiry Date   Storage Location  Stock Status  \
0        

In [7]:
print(df.isnull().sum())

Item Name             0
Supplier            178
Quantity              0
Cost Per Unit         0
Total Cost            0
Expiry Date           0
Storage Location      0
Stock Status          0
Order Date            0
Delivery Date       834
dtype: int64


In [8]:
df.shape

(8000, 10)

In [9]:
df.dropna
df.shape

(8000, 10)

In [10]:
# Fill missing supplier names with 'Unknown'
df['Supplier'].fillna('Unknown', inplace=True)

In [11]:
print(df.isnull().sum())

Item Name             0
Supplier              0
Quantity              0
Cost Per Unit         0
Total Cost            0
Expiry Date           0
Storage Location      0
Stock Status          0
Order Date            0
Delivery Date       834
dtype: int64


In [12]:
# Fill missing 'Delivery Date' with 'Not Delivered'
df["Delivery Date"].fillna("Not Delivered", inplace=True)

In [13]:
# If 'Total Cost' is NaN, recalculate it (Quantity * Cost Per Unit)
df["Total Cost"] = df["Quantity"] * df["Cost Per Unit"]

In [14]:
# Convert 'Expiry Date' column to datetime, replace invalid values with a future date
def fix_expiry_date(date):
    try:
        return pd.to_datetime(date)  # Convert valid dates
    except:
        return pd.to_datetime("2099-12-31")  # Replace invalid dates with a far future date

df["Expiry Date"] = df["Expiry Date"].apply(fix_expiry_date)


In [15]:
# Drop duplicate rows
df.drop_duplicates(inplace=True)

In [16]:
# Reset index and rename
df.reset_index(inplace=True)
df.rename(columns={"index": "id",
                   "Storage Location": "Location"}, inplace=True)

In [17]:
# Convert 'Item Name', 'Supplier', and 'Storage Location' to Title Case
df["Item Name"] = df["Item Name"].str.title()
df["Supplier"] = df["Supplier"].str.title()
df["Location"] = df["Location"].str.title()
df["Stock Status"] = df["Stock Status"].str.title()

In [18]:
df["Cost Per Unit"] = df["Cost Per Unit"].replace(r"[₹,]", "", regex=True).astype(float)
df["Total Cost"] = df["Total Cost"].replace(r"[₹,]", "", regex=True).astype(float)

# Round 'Cost Per Unit' and 'Total Cost' to 2 decimal places
df["Cost Per Unit"] = df["Cost Per Unit"].round(2)
df["Total Cost"] = df["Total Cost"].round(2)

In [19]:
# Fill missing Expiry Date and Order Date with a placeholder or forward fill
df["Expiry Date"].fillna("Not Available", inplace=True)
df["Order Date"].fillna("Not Available", inplace=True)

In [20]:
# Convert Order & delivery Date to datetime format
df["Order Date"] = pd.to_datetime(df["Order Date"], errors="coerce")
df["Delivery Date"] = pd.to_datetime(df["Delivery Date"], errors="coerce").dt.date
df["Delivery Date"] = df["Delivery Date"].fillna("Not Delivered")

In [21]:
#add column for delivery status
df["delivery_status"] = df["Delivery Date"].apply(lambda x: "Pending" if x == "Not Delivered" else "Delivered")
df["delivery_status"] = df["delivery_status"].astype("category")

In [22]:
print(df.dtypes)

id                          int64
Item Name                  object
Supplier                   object
Quantity                    int64
Cost Per Unit             float64
Total Cost                float64
Expiry Date        datetime64[ns]
Location                   object
Stock Status               object
Order Date         datetime64[ns]
Delivery Date              object
delivery_status          category
dtype: object


In [23]:
df["days_to_expiry"] = (df["Expiry Date"] - pd.Timestamp.today()).dt.days
df["expiry_status"] = df["days_to_expiry"].apply(lambda x: "Expired" if x < 0 else "Valid")

In [24]:
df.columns = df.columns.str.lower().str.replace(" ", "_")

In [25]:
df.sample(10)

,id,item_name,supplier,quantity,cost_per_unit,total_cost,expiry_date,location,stock_status,order_date,delivery_date,delivery_status,days_to_expiry,expiry_status
777,777,Necessary Diagnostic Tools,Carewell,63,430.83,27142.29,2027-06-11,Warehouse B,Available,2024-04-08,2024-04-18,Delivered,834,Valid
2674,2674,Health Personal Protective Equipment,Pharmaplus,47,251.91,11839.77,2028-02-16,Main Hospital,Available,2022-12-19,2023-11-03,Delivered,1084,Valid
5942,5942,But Surgical Equipment,Healthfirst,0,126.73,0.00,2027-01-21,Main Hospital,Out Of Stock,2023-07-28,2024-12-11,Delivered,693,Valid
5920,5920,As Medicine,Healthfirst,32,410.58,13138.56,2024-09-15,Main Hospital,Available,2024-01-07,2024-03-20,Delivered,-165,Expired
5910,5910,Reason Surgical Equipment,Healthfirst,216,456.75,98658.00,2025-03-28,Emergency Storage,Low Stock,2024-10-10,2025-02-12,Delivered,29,Valid
3805,3805,Yourself Medicine,Unknown,282,371.20,104678.40,2024-10-28,Emergency Storage,Low Stock,2022-06-12,2024-05-23,Delivered,-122,Expired
1493,1493,Sea Sanitizers,Medicorp,4,287.33,1149.32,2024-03-07,Warehouse B,Available,2023-02-09,2025-03-06,Delivered,-357,Expired
1550,1550,Staff Surgical Equipment,Medsupplies,248,80.33,19921.84,2027-12-26,Warehouse A,Out Of Stock,2024-03-22,2024-09-02,Delivered,1032,Valid
37,37,Political Medicine,Carewell,265,404.87,107290.55,2024-11-10,Emergency Storage,Out Of Stock,2023-12-05,Not Delivered,Pending,-109,Expired
3980,3980,Reflect Surgical Equipment,Medicorp,110,13.15,1446.50,2027-10-15,Emergency Storage,Available,2024-09-26,2025-02-01,Delivered,960,Valid


In [26]:
print(df.isnull().sum())

id                 0
item_name          0
supplier           0
quantity           0
cost_per_unit      0
total_cost         0
expiry_date        0
location           0
stock_status       0
order_date         0
delivery_date      0
delivery_status    0
days_to_expiry     0
expiry_status      0
dtype: int64


In [27]:
df.to_csv("cleaned.csv")

In [28]:
!pip install pyodbc

Defaulting to user installation because normal site-packages is not writeable


In [29]:
import pyodbc

cnxn = pyodbc.connect(
    "Driver={ODBC Driver 17 for SQL Server};"
    "Server=SHARVANIMAHADIK\\SQLEXPRESS;"
    "Database=hospital;"
    "Trusted_Connection=yes;"
)